# Physics Simulator
#### Experimental Project by Sahil Dev

In [1]:
import pygame
import pygame_menu
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.2.2


## Aesthetics

In [2]:
class Sound:
    circle_collide = None
    def init():
        Sound.circle_collide = pygame.mixer.Sound('sounds/collision1.wav')
        
class Color:
    class RGB(tuple):
        def __new__(cls, r, g, b):
            return tuple.__new__(cls, (int(r), int(g), int(b)))

        def __add__(self, other):
            return Color.RGB(self[0] + other[0], self[1] + other[1], self[2] + other[2])

        __radd__ = __add__

        def __mul__(self, other):
            return Color.RGB(self[0] * other, self[1] * other, self[2] * other)

        __rmul__ = __mul__

        def __truediv__(self, other):
            return Color.RGB(self[0] / other, self[1] / other, self[2] / other)

        __rtruediv__ = __truediv__
    
    black = RGB(0, 0, 0)
    red = RGB(255, 0, 0)
    green = RGB(0, 255, 0)
    blue = RGB(0, 0, 255)
    yellow = RGB(255, 255, 0)
    magenta = RGB(255, 0, 255)
    cyan = RGB(0, 255, 255)
    white = RGB(255, 255, 255)
    
    lavender = RGB(135, 129, 248)
    
    sky = RGB(100, 163, 236)
    ocean = RGB(80, 140, 200)
    
    rose = RGB(240, 100, 130)
    tomato = RGB(220, 80, 100)
    
    charcoal = RGB(44, 42, 48)
    darkgray = RGB(65, 60, 70)
    gray = RGB(100, 100, 100)
    lightgray = RGB(180, 180, 180)
    space = RGB(48, 45, 52)
    
    beige = RGB(215, 210, 200)
    
    
(Color.blue * 2.5 + Color.white * 3 + Color.red/10)/5.6

Color.blue + Color.red

(255, 0, 255)

## Basic Class Definitions

In [3]:
### Vec1D not used
# class Vec1D(np.ndarray):
#     def __new__(cls, magnitude):
#         if isinstance(magnitude, float):
#             return np.asarray(magnitude).view(cls)
#         raise Exception('Vec2D must be 1 dimensional.')

class Vec2D(np.ndarray):
    # overwrite np.ndarray new function to verify 2d
    def __new__(cls, iterable):
        if len(iterable) != 2:
            raise Exception('Vec2D must be 2 dimensional.')
        vec = np.asarray(iterable, dtype=np.longdouble).view(cls)
        return vec
    
    def intersectionPoint(a, u, b, v):
        a0, a1 = a[0], a[1]
        u0, u1 = u[0], u[1]
        b0, b1 = b[0], b[1]
        v0, v1 = v[0], v[1]

        # derived on paper
        s = ((b1 - a1 + (a0 - b0) * (v1/v0))/(u1 - u0 * (v1/v0)))
        return a + s * u

a, b = Vec2D([2, 1]), Vec2D([2, 4])
c = 2*a - b/2
c

Vec2D([3., 0.], dtype=float64)

In [4]:
class Acceleration(Vec2D):
    pass

class Velocity(Vec2D):
    # need to have self as leftmost value to preserve type
    def update(self, timestep, acceleration):
        return self + timestep * acceleration
    
    def bounceVelocity(self, vcalculated, bounciness, normal: Vec2D):
        # losable velocity
        lvf = normal * np.dot(vcalculated, normal)
        return vcalculated - lvf * (1 - bounciness)

class Position(Vec2D):
    def distance(self, point):
        return np.linalg.norm(self - point)
    
    def xdistance(self, point):
        return abs(self[0] - point[0])
    
    def ydistance(self, point):
        return abs(self[1] - point[1])
    
    # need to have self as leftmost value to preserve type
    def update(self, timestep, velocity, acceleration=Acceleration([0, 0])):
        return self + acceleration/2 * (timestep**2) + velocity * timestep

acc = Acceleration((0, -9.807))
vel = Velocity((0, 0))
pos = Position((0, 380))

# it takes just under 9 seconds to fall from the empire state building
for i in range(4):
    print(f't={3 * i}\tpos={pos}\t vel={vel}')
    pos = pos.update(3, vel, acc)
    vel = vel.update(3, acc)

t=0	pos=[  0. 380.]	 vel=[0. 0.]
t=3	pos=[  0.     335.8685]	 vel=[  0.    -29.421]
t=6	pos=[  0.    203.474]	 vel=[  0.    -58.842]
t=9	pos=[  0.     -17.1835]	 vel=[  0.    -88.263]


## Game Object Class Definitions

In [5]:
# represents any object within the game environment
class GameObject:
    def __init__(self):
        pass
    
    def update(self, timestep):
        raise NotImplementedError('Must override abstract method.')
    
    def checkCollision(self, gameobject, backward=False):
        raise NotImplementedError('Must override abstract method.')

    def onCollision(self, gameobject, data=None):
        raise NotImplementedError('Must override abstract method.')
        
class Viewport(GameObject):
    def fromPix(screendim, dim):
        width, height = screendim
        widthmeters, heightmeters = dim
        
        if width is None or height is None:
            raise Exception('Must input screen dimensions.')
        if widthmeters is None and heightmeters is None:
            raise Exception('Must have at least one dimension defined.')
        
        aspectratio = width/height
        if widthmeters is None:
            widthmeters = heightmeters * aspectratio
        if heightmeters is None:
            heightmeters = widthmeters / aspectratio
        
        low = Position((-widthmeters/2, 0))
        high = Position((widthmeters/2, heightmeters))
        
        return Viewport(low, high)
        
    def __init__(self, low: Position, high: Position):
        super().__init__()
        self.low = low
        self.high = high
        
    def width(self):
        return (self.high - self.low)[0]
    
    def height(self):
        return (self.high - self.low)[1]
    
    # might need to change if viewport is allowed to change
    def update(self, timestep):
        pass
    
    # use other gameobject collision function
    def checkCollision(self, gameobject, backward=False):
        if isinstance(gameobject, Viewport):
            return False
            # raise Exception('Attempted collision check with two Viewports.')
        return gameobject.checkCollision(self, backward=backward)
    
    # don't do anything to the viewport when colliding with an object
    def onCollision(self, gameobject, data=None):
        pass

In [6]:
# constants

GRAVITY = Acceleration((0, -9.807))
ORIGIN = Position((0, 0))
NO_VELOCITY = Velocity((0, 0))
NO_ACCELERATION = Acceleration((0, 0))

# x = -10m to +10m, y = 0m to +20m
DEFAULT_VIEWPORT = Viewport(Position((-10, 0)), Position((10, 20)))

In [7]:
# represents an object that responds to physics
class PhysicsObject(GameObject):
    def __init__(self, mass, position, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__()
        self.mass = mass
        self.position = position
        self.velocity = velocity
        self.gravity = gravity
        self.bounciness = bounciness

    def update(self, timestep):
        self.position = self.position.update(timestep, self.velocity, self.gravity)
        self.velocity = self.velocity.update(timestep, self.gravity)
        self.gravity = self.gravity # no-op
        
class Circle(PhysicsObject):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, gravity=GRAVITY, 
                 bounciness=1, color=Color.gray, shadowcolor=Color.charcoal):
        super().__init__(mass, center, velocity=velocity, gravity=gravity, bounciness=bounciness)
        self.radius = radius
        self.color = color
        self.shadowcolor = shadowcolor
    
    def checkCollision(self, gameobject, backward=False):
        collision, data = None, None
        
        # collision with screen border
        if isinstance(gameobject, Viewport):
            vf = self.velocity
            bounciness = self.bounciness
            
            if backward:
                vf = -vf
                bounciness = 1/bounciness

            vport = gameobject
            
            highx_collision = (vport.high[0] - self.position[0]) < self.radius and vf[0] > 0
            if highx_collision:
                vf = Velocity((-bounciness * vf[0], vf[1]))
                
            highy_collision = (vport.high[1] - self.position[1]) < self.radius and vf[1] > 0
            if highy_collision:
                vf = Velocity((vf[0], -bounciness * vf[1]))
                
            lowx_collision = (self.position[0] - vport.low[0]) < self.radius and vf[0] < 0
            if lowx_collision:
                vf = Velocity((-bounciness * vf[0], vf[1]))
                
            lowy_collision = (self.position[1] - vport.low[1]) < self.radius and vf[1] < 0
            if lowy_collision:
                vf = Velocity((vf[0], -bounciness * vf[1]))
            
            if backward:
                vf = -vf
            
            collision = (lowx_collision or lowy_collision or highx_collision or highy_collision)
            
            data = {
                self: {
                    'vel': vf
                },
                'sound': None
            }
            
            return collision, data
        
        # collision with another circle
        elif isinstance(gameobject, Circle):
            p1, p2 = self.position, gameobject.position
            v1, v2 = self.velocity, gameobject.velocity
            
            if backward:
                v1 = -v1
                v2 = -v2
            
            dist = p1.distance(p2)
            collision = (dist < self.radius + gameobject.radius) and \
                (p2 - p1).dot(v2 - v1) < 0
            
            # TODO: treat balls as more than points!
            # TODO: incorporate energy loss? - maybe not necessary
            if collision:
                # normal = (p2 - p1)/p1.distance(p2)
                # v1i = self.velocity.bounceVelocity(v1, 1, normal)
                # v2i = self.velocity.bounceVelocity(v2, 1, normal)
                
                # solve for final velocities (elastic collision)
                # equation 1: v1i + v1f = v2i + v2f
                # equation 2: m1v1i + m2v2i = m1v1f + m2v2f
                
                m1, m2 = self.mass, gameobject.mass
                v1i, v2i = v1, v2
                
                # useful constants
                D = v2i - v1i
                P = m1 * v1i + m2 * v2i # sum(momentum)
                
                # final velocities
                v2f = (P - m1 * D)/(m1 + m2)
                v1f = v2f + D
                
                # normal direction, needed for bounciness calculation
                # normal = (p2 - p1)/p1.distance(p2)
                
                if backward:
                    v1f = -v1f
                    v2f = -v2f
                
                data = {
                    self: {
                        'vel': v1f,
                    }, 
                    gameobject: {
                        'vel': v2f,
                    },
                    'sound': Sound.circle_collide
                }
                
                # idea: prevent them getting stuck together by finding
                #       new centers that don't overlap
                # this is a bad idea - harder than checking mvmt direction and less reversible
                
            return collision, data
        
        else:
            pass
    
    def onCollision(self, gameobject, data=None):
        if isinstance(gameobject, Viewport):
            data = data[self]
            self.velocity = data['vel']

        elif isinstance(gameobject, Circle):
            data = data[self]
            self.velocity = data['vel']

        else:
            pass

    def isInside(self, position):
        return self.position.distance(position) <= self.radius
    
    def withinDistance(self, position, radius):
        return self.position.distance(position) <= self.radius + radius
    
    def toBall(self, cls):
        return cls(self.mass, self.position, self.radius, velocity=self.velocity,
                   bounciness=self.bounciness, shadowcolor=self.shadowcolor)
    
    def toLightSource(self):
        return LightSource(self.mass, self.position, self.radius, velocity=self.velocity,
                gravity=self.gravity, bounciness=self.bounciness, shadowcolor=self.shadowcolor)

In [8]:
class GravityBall(Circle):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, 
                 bounciness=1, shadowcolor=Color.charcoal):
        super().__init__(mass, center, radius, velocity=velocity, gravity=GRAVITY, 
                 bounciness=bounciness, color=Color.tomato, shadowcolor=shadowcolor)

class AntiGravityBall(Circle):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, 
                 bounciness=1, shadowcolor=Color.charcoal):
        super().__init__(mass, center, radius, velocity=velocity, gravity=-GRAVITY, 
                 bounciness=bounciness, color=Color.ocean, shadowcolor=shadowcolor)

class NoGravityBall(Circle):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY,
                 bounciness=1, shadowcolor=Color.charcoal):
        super().__init__(mass, center, radius, velocity=velocity, gravity=NO_ACCELERATION, 
                 bounciness=bounciness, color=Color.lavender, shadowcolor=shadowcolor)

class LightSource(Circle):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, gravity=-GRAVITY,
                 bounciness=1, shadowcolor=Color.charcoal):
        super().__init__(mass, center, radius, velocity=velocity, gravity=gravity, 
                 bounciness=bounciness, color=Color.beige, shadowcolor=shadowcolor)

    # return the three bounding lines of the shadow (incl diameter of circle)
    def calculateShadows(self, gameobject):
        lightpos = self.position
        c, d = lightpos
        r1 = self.radius

        if isinstance(gameobject, LightSource):
            return None
        elif isinstance(gameobject, Circle):
            circlepos = gameobject.position
            a, b = circlepos
            r0 = gameobject.radius
            
            if r1 != r0:
                xp = (c * r0 - a * r1)/(r0 - r1)
                yp = (d * r0 - b * r1)/(r0 - r1)
                
                # calculate points on circle object
                A = (r0 ** 2) * (xp - a)
                B = r0 * (yp - b)
                C = np.sqrt((xp - a)**2 + (yp - b)**2 - r0**2)
                D = (xp - a)**2 + (yp - b)**2
                # x values on circle object
                x1 = (A + B * C)/D + a
                x2 = (A - B * C)/D + a
                
                E = (r0 ** 2) * (yp - b)
                F = r0 * (xp - a)
                # y values on circle object
                # need - for first solution, + for second (bc it corresponds to x1 and x2)
                y1 = (E - F * C)/D + b
                y2 = (E + F * C)/D + b
                
                # sanity check, one of these should be 1
                s1 = (b - y1) * (yp - y1)/((x1 - a) * (x1 - xp))
                s2 = (b - y2) * (yp - y2)/((x1 - a) * (x1 - xp))
                
                # calculate points on light source object
                G = (r1 ** 2) * (xp - c)
                H = r1 * (yp - d)
                I = np.sqrt((xp - c)**2 + (yp - d)**2 - r1**2)
                J = (xp - c)**2 + (yp - d)**2
                # x values on light source object
                x3 = (G + H * I)/J + c
                x4 = (G - H * I)/J + c
                
                K = (r1 ** 2) * (yp - d)
                L = r1 * (xp - c)
                # y values on light source object
                # need - for first solution, + for second (bc it corresponds to x3 and x4)
                y3 = (K - L * I)/J + d
                y4 = (K + L * I)/J + d
                
                # sanity check, one of these should be 1
                s3 = (d - y3) * (yp - y3)/((x3 - c) * (x3 - xp))
                s4 = (d - y4) * (yp - y4)/((x3 - c) * (x3 - xp))
                
                return (Vec2D((x1, y1)), Vec2D((x2, y2))), (Vec2D((x3, y3)), Vec2D((x4, y4)))

            else:
                # TODO: calculate shadows for same radius
                pass

## Game Logic

In [9]:
class Game:
    def __init__(self, viewport, objects, screendim, lightsources=None):
        if lightsources is None:
            lightsources = []
            
        self.screendim = screendim
        self.viewport = viewport
        self.lightsources = lightsources
        self.objects = objects
        self.time = 0
        
    def update(self, timestep):
        self.time += timestep
        
        self.viewport.update(timestep) # probably won't ever do anything
        backward = (timestep < 0)
        
        # get all objects
        objects = self.lightsources + self.objects
        
        # run update step on all objects
        for gameobject in objects:
            gameobject.update(timestep)
        
        # run collision check against viewport
        for gameobject in objects:
            collision, data = gameobject.checkCollision(self.viewport, backward=backward)
            if collision:
                gameobject.onCollision(self.viewport, data)
        
        new_collisions = set()
        # run collision check on all object pairs
        for index, gameobject1 in enumerate(objects):
            for gameobject2 in self.objects[index + 1:]:
            
                # check only one direction, should be symmetric (we'll see about that lol)
                collision, data = gameobject1.checkCollision(gameobject2, backward=backward)
                if collision:
                    # run collision method for each object in collision
                    gameobject1.onCollision(gameobject2, data)
                    gameobject2.onCollision(gameobject1, data)
                    sound = data['sound']
                    if sound is not None:
                        pygame.mixer.Sound.play(sound)

    def pixFromVec(self, vec: Vec2D, centered=True):
        widthpix, heightpix = self.screendim # screen dimensions, in pixels
        low, high = self.viewport.low, self.viewport.high # viewport positions, in meters
        widthmeters, heightmeters = high - low # viewport dimensions, in meters
        
        if centered:
            # find distance from top left corner instead
            distfromleft = vec[0] - low[0]
            distfromtop = high[1] - vec[1]
            vec = Vec2D((distfromleft, distfromtop))
        
        # convert distance to pixels
        scale = Vec2D((widthpix / widthmeters, heightpix / heightmeters))
        return vec * scale
    
    def vecFromPix(self, pixloc, centered=True):
        width, height = self.screendim # screen dimensions, in pixels
        low, high = self.viewport.low, self.viewport.high # viewport positions, in meters
        widthmeters, heightmeters = high - low # viewport dimensions, in meters
        
        # convert pixels to distance
        vec = Vec2D((pixloc[0], height - pixloc[1]))
        scale = Vec2D((widthmeters / width, heightmeters / height))
        vec = vec * scale
        
        if centered:
            vec = vec + low
        
        return vec
    
    # TODO: don't draw things that are out of frame (we can still keep doing physics on them tho)
    # necessary change dt overflow issues in generating the rectangle
    def draw(self, window, gameobject):
        if isinstance(gameobject, Circle):
            center = self.pixFromVec(gameobject.position, centered=True)
            radius = self.pixFromVec(Vec2D((gameobject.radius, gameobject.radius)), centered=False)
            left, top = center - radius
            w, h = radius * 2
            
            # circle
            # TODO: don't draw if outside frame
            rect = pygame.Rect(left, top, w, h)
            pygame.draw.ellipse(window, gameobject.color, rect)

        else:
            pass
        
    def drawAll(self, window):
        for gameobject in self.objects:
            for lightsource in self.lightsources:
                self.drawShadow(window, gameobject, lightsource)
                # TODO: draw shadows
                self.draw(window, lightsource)
            self.draw(window, gameobject)
    
    def drawShadow(self, window, gameobject, lightsource: LightSource):
        if isinstance(gameobject, Circle):
            (cp0, cp1), (lp0, lp1) = lightsource.calculateShadows(gameobject)
            
            pygame.draw.line(window, Color.beige, self.pixFromVec(lp0, centered=True),
                             self.pixFromVec(cp0, centered=True), width=3)
            pygame.draw.line(window, Color.beige, self.pixFromVec(lp1, centered=True),
                             self.pixFromVec(cp1, centered=True), width=3)
                
            l0, l1 = cp0 - lp0, cp1 - lp1
            l0, l1 = l0/np.linalg.norm(l0), l1/np.linalg.norm(l1)
            
            # define walls as vector pointing parallel to wall
            w0 = Vec2D((0, 1)) # right wall
            w0i = Vec2D((self.viewport.high[0], self.viewport.low[1]))
            
            w1 = Vec2D((1, 0)) # ceiling
            w1i = Vec2D((self.viewport.low[0], self.viewport.high[1]))
            
            w2 = Vec2D((0, 1)) # left wall
            w2i = Vec2D((self.viewport.low[0], self.viewport.low[0]))
            
            w3 = Vec2D((1, 0)) # floor
            w3i = Vec2D((self.viewport.low[0], self.viewport.low[1]))
            
            # find which walls each line collides with
            p00 = Vec2D.intersectionPoint(cp0, l0, w0i, w0)
            p01 = Vec2D.intersectionPoint(cp0, l0, w1i, w1)
            p02 = Vec2D.intersectionPoint(cp0, l0, w2i, w2)
            p03 = Vec2D.intersectionPoint(cp0, l0, w3i, w3)
            
            if np.dot(l0, p00 - cp0) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp0, centered=True),
                                 self.pixFromVec(p00, centered=True), width=3)
            if np.dot(l0, p01 - cp0) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp0, centered=True),
                                 self.pixFromVec(p01, centered=True), width=3)
            if np.dot(l0, p02 - cp0) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp0, centered=True),
                                 self.pixFromVec(p02, centered=True), width=3)
            if np.dot(l0, p03 - cp0) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp0, centered=True),
                                 self.pixFromVec(p03, centered=True), width=3)
            
            # find which walls each line collides with
            p10 = Vec2D.intersectionPoint(cp1, l1, w0i, w0)
            p11 = Vec2D.intersectionPoint(cp1, l1, w1i, w1)
            p12 = Vec2D.intersectionPoint(cp1, l1, w2i, w2)
            p13 = Vec2D.intersectionPoint(cp1, l1, w3i, w3)
            
            if np.dot(l1, p10 - cp1) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp1, centered=True),
                                 self.pixFromVec(p10, centered=True), width=3)
            if np.dot(l1, p11 - cp1) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp1, centered=True),
                                 self.pixFromVec(p11, centered=True), width=3)
            if np.dot(l1, p12 - cp1) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp1, centered=True),
                                 self.pixFromVec(p12, centered=True), width=3)
            if np.dot(l1, p13 - cp1) > 0:
                pygame.draw.line(window, Color.cyan, self.pixFromVec(cp1, centered=True),
                                 self.pixFromVec(p13, centered=True), width=3)
                
            
        else:
            pass
        
    def drawAllShadows(self, window):
        for gameobject in self.objects:
            self.drawShadow(window, gameobject)

screendim = (1920, 1200)
vport = Viewport.fromPix(screendim, (10, None))
# vport = Viewport(Position((0, 0)), Position((1920, 1200)))
g = Game(vport, [], screendim)

# origin is at center bottom of screen
g.pixFromVec(Vec2D((0, 0)), centered=True)

Vec2D([ 960., 1200.], dtype=float64)

In [10]:
g = Game(vport, [Circle(10, Position((1, 3)), 1), Circle(10, Position((-2, 2)), 1)], screendim)
for gameobject in g.objects:
    print(f'{type(gameobject)}: position={gameobject.position}, velocity={gameobject.velocity}')

<class '__main__.Circle'>: position=[1. 3.], velocity=[0. 0.]
<class '__main__.Circle'>: position=[-2.  2.], velocity=[0. 0.]


## Pygame GUI

In [11]:
DEFAULT_DIM = (40, None)
DEFAULT_MASS = 10
DEFAULT_RADIUS = 1
DEFAULT_BOUNCINESS = 0.95
FAST_FORWARD_SPEED = 9
NORMAL_SPEED = 3
SLOW_MOTION_SPEED = 1

def getMousePos(game):
    return Position(game.vecFromPix(pygame.mouse.get_pos(), centered=True))

def radiusToMass(radius):
    return DEFAULT_MASS * (radius/DEFAULT_RADIUS)**2

def create(window, clock=None, fps=60, game=None):
    screendim = window.get_size()
    if clock is None:
        clock = pygame.time.Clock()
    if game is None:
        game = Game(Viewport.fromPix(screendim, DEFAULT_DIM), [], screendim)
    viewport = game.viewport
    
    # setup initial parameters
    mass = DEFAULT_MASS
    radius = DEFAULT_RADIUS
    bounciness = DEFAULT_BOUNCINESS
    
    # placeholder where the mouse is
    def getPlaceholder():
        pos = getMousePos(game)
        return Circle(mass, pos, radius, bounciness=bounciness, 
           velocity=Velocity(np.random.normal(loc=0, scale=viewport.width()/4, size=(2,))))
    
    placeholder = getPlaceholder()
    
    # TODO: keep the circles in sorted order based on size
    # and let the larger ones cast shadows on smaller ones
    running = True
    while running:
        # draw current frame
        window.fill(Color.space)
        
        # check if mouse within radius distance of any circles
        canadd, insideptr = True, -1
        mousepos = getMousePos(game)
        
        # TODO: also check walls
        for i, gameobject in enumerate(game.objects):
            if gameobject.withinDistance(mousepos, radius):
                canadd = False
            if gameobject.isInside(mousepos):
                insideptr = i
        
        # update placeholder circle
        if canadd:
            placeholder.position = mousepos
        
        # draw frame
#         game.drawAllShadows(window)
#         if canadd:
#             game.drawShadow(window, placeholder)
        
        game.drawAll(window)
        if canadd:
            game.draw(window, placeholder)
        
        # check input
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            elif event.type == pygame.KEYDOWN:
                # exit
                if event.key in [pygame.K_q, pygame.K_ESCAPE]:
                    running = False
                    
                # clear board
                if event.key in [pygame.K_c]:
                    game.objects = []
                    game.time = 0
                    
            elif event.type == pygame.MOUSEBUTTONUP:
                # TODO: consider allowing reclick to replace existing ball 
                # with new one of different type, weigh costs and benefits
                
                if event.button == 1: # left click, normal gravity
                    if canadd:
                        game.objects.append(placeholder.toBall(GravityBall))
                        placeholder = getPlaceholder()
                    elif insideptr > -1:
                        game.objects.pop(insideptr)
                        
                elif event.button == 2: # middle click
                    if canadd:
                        game.objects.append(placeholder.toBall(NoGravityBall))
                        placeholder = getPlaceholder()
                        
                elif event.button == 3: # right click
                    if canadd:
                        game.objects.append(placeholder.toBall(AntiGravityBall))
                        placeholder = getPlaceholder()
            
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4: # scroll down
                    radius *= 0.97
                    placeholder.radius = radius
                elif event.button == 5: # scroll up
                    radius /= 0.97
                    placeholder.radius = radius
                placeholder.mass = radiusToMass(radius)
                
        
        # update
        pygame.display.update()
        clock.tick(fps)
    
    return game

In [12]:
# defined above
# FAST_FORWARD_SPEED = 9
# NORMAL_SPEED = 3
# SLOW_MOTION_SPEED = 1

def getDefaultGame(screendim):
    return Game(Viewport.fromPix(screendim, DEFAULT_DIM),
            [GravityBall(10, Position((-3, 4)), 1, bounciness=0.97, velocity=Velocity((3, -2))),
             AntiGravityBall(10, Position((2, 6)), 1, bounciness=0.9, velocity=Velocity((-6, 6))),
             GravityBall(2.5, Position((8, 3)), 0.5, bounciness=0.8, velocity=Velocity((-15, 15))),
             NoGravityBall(4.9, Position((-6, 10)), 0.7, bounciness=0.85, velocity=Velocity((15, -15))),
             AntiGravityBall(0.9, Position((8, 10)), 0.3, bounciness=0.9, velocity=Velocity((10, 30)))
            ], screendim, lightsources=[
                LightSource(32.4, Position((-15, 15)), 1.8, velocity=Velocity((6, -7)),
                           bounciness=0.98, gravity=NO_ACCELERATION)
            ])

def start(window, clock=None, fps=60, game=None, experimentaltime=False):
    
    # set up delta time
    deltatime = None
    if experimentaltime:
        fakefps = np.power(2, np.ceil(np.log2(fps)))
        deltatime = (1/fakefps) / NORMAL_SPEED
        print(f'Using delta time of 1/{fakefps}s per frame at {fps} fps')
    else:
        deltatime = (1/fps) / NORMAL_SPEED
        print(f'Using delta time of 1/{fps}s per frame at {fps} fps')
    
    # basic setup
    screendim = window.get_size()
    if clock is None:
        clock = pygame.time.Clock()
    if game is None:
        game = getDefaultGame(screendim)
    viewport = game.viewport
    
    # text for game state/speed
    speed = NORMAL_SPEED
    speedfont = pygame.font.SysFont('Trebuchet MS', 60, bold=True)
    timefont = pygame.font.SysFont('Trebuchet MS', 40, bold=True)
    speedlocation = (screendim[0] - 140, screendim[1] - 100)
    timelocation = (40, screendim[1] - 80)
    
    pausedtext = speedfont.render('||', False, Color.white)
    forwardtextslow = speedfont.render('  >', False, Color.white)
    backwardtextslow = speedfont.render('  <', False, Color.white)
    forwardtext = speedfont.render(' >>', False, Color.white)
    backwardtext = speedfont.render(' <<', False, Color.white)
    forwardtextff = speedfont.render('>>>', False, Color.white)
    backwardtextff = speedfont.render('<<<', False, Color.white)
    
    # speed stuff
    fdown, sdown = False, False
    # 0 = paused, 1 = forward, 2 = backward (, -1 = stopped)
    state, prev_state, step = 0, 1, 0
    while state >= 0:
        # draw current frame
        window.fill(Color.space)
#         game.drawAllShadows(window)
        game.drawAll(window)

        # check input
        step = 0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                state = -1
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_f:
                    fdown = False
                if event.key == pygame.K_s:
                    sdown = False
                    
            elif event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_q, pygame.K_ESCAPE]:
                    state = -1
                elif event.key in [pygame.K_SPACE, pygame.K_k]:
                    if state == 0:
                        state = prev_state
                    else:
                        prev_state = state
                        state = 0
                elif event.key == pygame.K_l:
                    state = 1
                elif event.key == pygame.K_j:
                    state = 2
                    
                elif event.key == pygame.K_COMMA:
                    step = -1
                elif event.key == pygame.K_PERIOD:
                    step = 1
                    
                elif event.key == pygame.K_f:
                    fdown = True
                elif event.key == pygame.K_s:
                    sdown = True
        
        # set speed based on f and s being held down
        if (fdown and sdown) or not (fdown or sdown):
            speed = NORMAL_SPEED
        elif fdown:
            speed = FAST_FORWARD_SPEED
        elif sdown:
            speed = SLOW_MOTION_SPEED
        
        # print speed in bottom right
        if state == 0:
            window.blit(pausedtext, speedlocation)
        elif state == 1:
            if speed == NORMAL_SPEED:
                window.blit(forwardtext, speedlocation)
            if speed == FAST_FORWARD_SPEED:
                window.blit(forwardtextff, speedlocation)
            if speed == SLOW_MOTION_SPEED:
                window.blit(forwardtextslow, speedlocation)
        elif state == 2:
            if speed == NORMAL_SPEED:
                window.blit(backwardtext, speedlocation)
            if speed == FAST_FORWARD_SPEED:
                window.blit(backwardtextff, speedlocation)
            if speed == SLOW_MOTION_SPEED:
                window.blit(backwardtextslow, speedlocation)
        
        # print time in bottom left
        window.blit(timefont.render(f'{np.round(game.time, 2)} s', False, Color.white),
                    timelocation)
            
        # update
        pygame.display.update()
        clock.tick(fps)
        
        # TODO: find if there is any update amount that reduces floating point error issues
        for i in range(speed):
            if state == 0:
                if step != 0:
                    game.update(step * deltatime)
            elif state == 1:
                game.update(deltatime)
            elif state == 2:
                game.update(-deltatime)

In [13]:
def main_menu(window):
    game = None
    clock = pygame.time.Clock()
    FPS = 64
    
    def start_game():
        start(window, clock=clock, fps=FPS, game=game)
    
    def create_game():
        nonlocal game
        game = create(window, clock=clock, fps=FPS, game=game)
    
    menu = pygame_menu.Menu('Physics Simulator', 400, 400,
                            theme=pygame_menu.themes.THEME_DARK)
    menu.add.button('Play', start_game)
    menu.add.button('Create', create_game)
    menu.add.button('Quit', pygame_menu.events.EXIT)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return

    menu.mainloop(window)

In [14]:
def main():
    # pygame initialization
    print('Starting pygame window...')
    
    pygame.mixer.init(48000, -16, 1, 1024)
    pygame.init()
    Sound.init()
    
    window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN) # window size 0,0 for full screen
    pygame.display.set_caption("Physics Simulator")

    print('Starting main menu...')
    main_menu(window)

In [ ]:
main()

Starting pygame window...
Starting main menu...
Using delta time of 1/64s per frame at 64 fps


<ipython-input-9-35d568e531f6>:105: DeprecationWarning: an integer is required (got type numpy.longdouble).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pygame.draw.line(window, Color.beige, self.pixFromVec(lp0, centered=True),
<ipython-input-9-35d568e531f6>:107: DeprecationWarning: an integer is required (got type numpy.longdouble).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pygame.draw.line(window, Color.beige, self.pixFromVec(lp1, centered=True),
<ipython-input-3-aa5a15dd14e2>:23: RuntimeWarning: divide by zero encountered in longdouble_scalars
  s = ((b1 - a1 + (a0 - b0) * (v1/v0))/(u1 - u0 * (v1/v0)))
<ipython-input-3-aa5a15dd14e2>:23: RuntimeWarning: invalid value encountered in longdouble_scalars
  s = ((b1 - a1 + (a0 - b0) * (v1/v0))/(u1 - u0 * (v1/v0)))
<ipython-input-9-35d568e531f6>:142: DeprecationWarning: an integer is required (got typ